# Common Base Amplifier
* Low input impedance.  Matching network converts to a lower voltage, which is a drawback compared to the common emitter or cascode amplifier.
* high output impedance, so can be loaded with a parallel resonant circuit.
* No Miller effect

For the calculations, the "Analysis and Design of Analog Integrated Circuits, Fourth Edition" by Gray and Meyer is used.

# Requirements


In [11]:
# Requirement
G_T_dB = 15 # Transducer gain in dB
Vcc = 7 # Supply voltage
V_T = 26e-3 # Thermal voltage

The power gain for a CB-amplifier is given by: g_m * R_C / 4.  Moreover, I_c * R_C must remain less than V_CC.  This limits the gain for a resistive load.  For a parallel RLC resonant load, the resistor is DC-short circuited by the inductor and higher gain can be achieved.

# Transistors
* [MMBT5179, CMUT5179 (Mouser)]()
* BFU550

# 1. Bias circuit
## Stability
[Self Bias or Potential Divider Bias Circuit](https://www.eeeguide.com/self-bias-or-potential-divider-bias-circuit/)

MegaWattKS : set V_{E} in the range of 0.5V to 1V
Art of Electronics, §2.13 : set R_{E} as 1/10 of R_{C}

## DC Analysis

In [12]:
# Biasing

def calc_Rc(Vcc, Ic, G_T_dB):
    G_T = 10**(G_T_dB/10)
    Ic_Rc = V_T * 4 * G_T
    if Ic_Rc > Vcc:
        print("Ic * Rc > Vcc, impossible with resistive load.  Use a tuned parallel LC circuit as the load.")
    Rc = Ic_Rc / Ic # Collector resistor
    return Rc

def calc_bias(Ic, Vbe, V_E, hfe):
    # Calculate biasing resistors
    Re = V_E / Ic # Emitter resistor
    print("\tEmitter resistor: {:.2f} ohm".format(Re))
    Ib = Ic / hfe # Base current
    V_R2 = Vbe + V_E # Voltage over R2
    I_R2 = 10 * Ib # Current through R2 for stable biasing
    R2 = V_R2 / I_R2 # Base resistor
    print("\tBottom bias resistor: {:.2f} ohm".format(R2))
    V_R1 = Vcc - V_R2 # Voltage over R1
    I_R1 = I_R2 + Ib # Current through R1
    R1 = V_R1 / I_R1 # Top bias resistor
    print("\tTop bias resistor: {:.2f} ohm".format(R1))
    return R1, R2, Re

def stability_check(R1, R2, Re, hfe):
    R_th = R1 * R2 / (R1 + R2) # Thevenin resistance
    S = (hfe + 1) * (Re + R_th) / ((hfe+1) * Re + R_th) # Stability factor
    print("\tStability factor: {:.2f}".format(S))
    if S < 10:
        print("\tGood biasing")
    elif S < 20:
        print("\tAcceptable biasing")
    else:
        print("\tUnstable biasing")

Vbe = 0.8 # Base-emitter voltage
V_E = 1 # Assumed voltage over emitter resistor
hfe = 100 # Current gain
Ic = [1e-3, 2e-3, 3e-3] # Collector current

for i in Ic:
    print("Biasing for Ic = {:.2f} mA".format(i*1e3))
    [R1, R2, Re] = calc_bias(i, Vbe, V_E, hfe)
    stability_check(R1, R2, Re, hfe)

# Actual values for the resistors

# Ic = 1 mA
# Re = 1e3
# R1 = 47e3
# R2 = 18e3

# Ic = 2 mA
# Re = 510
# R1 = 24e3
# R2 = 10e3

# Ic = 3 mA
# Re = 300
# R1 = 15e3
# R2 = 5.6e3

# Calculate helper variables
## Transconductance (gm)
gm = [x / V_T  for x in Ic] # Transconductance


Biasing for Ic = 1.00 mA
	Emitter resistor: 1000.00 ohm
	Bottom bias resistor: 18000.00 ohm
	Top bias resistor: 47272.73 ohm
	Stability factor: 12.43
	Acceptable biasing
Biasing for Ic = 2.00 mA
	Emitter resistor: 500.00 ohm
	Bottom bias resistor: 9000.00 ohm
	Top bias resistor: 23636.36 ohm
	Stability factor: 12.43
	Acceptable biasing
Biasing for Ic = 3.00 mA
	Emitter resistor: 333.33 ohm
	Bottom bias resistor: 6000.00 ohm
	Top bias resistor: 15757.58 ohm
	Stability factor: 12.43
	Acceptable biasing


# 2. Output resistance

In [13]:
VA = 100 # Early voltage
ro = [VA / x_Ic for x_Ic in Ic] # (1.91, 1.13 & 1.114)
Rc = [calc_Rc(Vcc, x, G_T_dB) for x in Ic] # Collector resistor
Rc_actual = [3.3e3, 1.6e3, 1.1e3] # Actual collector resistor

# Output resistance Rout = Rc || ro
Rout = [x_ro * y_Rc / (x_ro + y_Rc) for x_ro, y_Rc in zip(ro, Rc)] # (3.32)
Rout_actual = [x_ro * y_Rc / (x_ro + y_Rc) for x_ro, y_Rc in zip(ro, Rc_actual)] # (3.32)

for i, x in enumerate(Ic):
    print("Output resistance for Ic = {:.2f} mA: {:.2f} ohm".format(x*1e3, Rout[i]))
    print("Actual output resistance for Ic = {:.2f} mA: {:.2f} ohm".format(x*1e3, Rout_actual[i]))

Output resistance for Ic = 1.00 mA: 3184.05 ohm
Actual output resistance for Ic = 1.00 mA: 3194.58 ohm
Output resistance for Ic = 2.00 mA: 1592.03 ohm
Actual output resistance for Ic = 2.00 mA: 1550.39 ohm
Output resistance for Ic = 3.00 mA: 1061.35 ohm
Actual output resistance for Ic = 3.00 mA: 1064.86 ohm


# 3. Unloaded Voltage Gain
<figure>
    <img src="./doc/simulation/2_Av_no_load.png" width="100%">
    <figcaption>Common Base Amplifier with no load for bias settings of 1mA, 2mA and 3mA.</figcaption>
</figure>

In [14]:
# Unloaded voltage gain Av_no_load = gm * Rout
import math
Av_no_load = [x_gm * y_Rout for x_gm, y_Rout in zip(gm, Rout)] # Voltage gain, unloaded
Av_no_load_dB = [20 * math.log10(x_Av_no_load) for x_Av_no_load in Av_no_load]

for i, x in enumerate(Ic):
    print("Unloaded voltage gain for Ic = {:.2f} mA: {:.2f} dB".format(x*1e3, Av_no_load_dB[i]))

Unloaded voltage gain for Ic = 1.00 mA: 41.76 dB
Unloaded voltage gain for Ic = 2.00 mA: 41.76 dB
Unloaded voltage gain for Ic = 3.00 mA: 41.76 dB


# 4. Input impedance
## Input resistance

In [15]:
Rin = [1 / x_gm for x_gm in gm] # Input resistance
for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    print("\tTransconductance: {:.2f} S".format(gm[i]))
    print("\tInput resistance: {:.2f} ohm".format(Rin[i]))

For Ic = 1.00 mA
	Transconductance: 0.04 S
	Input resistance: 26.00 ohm
For Ic = 2.00 mA
	Transconductance: 0.08 S
	Input resistance: 13.00 ohm
For Ic = 3.00 mA
	Transconductance: 0.12 S
	Input resistance: 8.67 ohm


## Input capacitance

In [16]:
import math

Cµ = 1e-12 # Ccb
fT = 800e6 # Transition frequency

# Frequency pole due to capacitance on the input
Cπ = [x_gm / (2 * math.pi * fT) - Cµ for x_gm in gm] # [Finding Cbe](https://youtu.be/m9X0mfg_8lQ?t=3378)
for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    print("\tInput capacitance: {:.2e} F".format(Cπ[i]))
    f_p1 = gm[i] / (2 * math.pi * Cπ[i]) #
    print("\tCutoff frequency due to capacitance at the input: {:.2e} Hz".format(f_p1))

For Ic = 1.00 mA
	Input capacitance: 6.65e-12 F
	Cutoff frequency due to capacitance at the input: 9.20e+08 Hz
For Ic = 2.00 mA
	Input capacitance: 1.43e-11 F
	Cutoff frequency due to capacitance at the input: 8.56e+08 Hz
For Ic = 3.00 mA
	Input capacitance: 2.20e-11 F
	Cutoff frequency due to capacitance at the input: 8.36e+08 Hz


## Simulation
<figure>
    <img src="./doc/simulation/3_input_impedance.png" width="100%">
    <figcaption>Input impedance (shown as voltage over a 1A current source) for bias settings of 1mA, 2mA and 3mA.</figcaption>
</figure>

# 5. Input Matching network
## Calculation

In [17]:
def calc_L_match(f, Rp, Rs):
    # f : frequency where the matching is done
    # Rp : load resistance on the parallel side
    # Rs : source resistance on the series side
    # Rp will always be higher than Rs
    print("\tLoad resistance: {:.2f} ohm".format(Rp))
    print("\tSource resistance: {:.2f} ohm".format(Rs))
    ω = 2 * math.pi * f
    Q = math.sqrt(Rp / Rs - 1)
    print("\tQ: {:.2f}".format(Q))
    Xs = Q * Rs # series reactance
    Xp = Rp / Q # parallel reactance
    C = 1 / (Xs * ω) # we choose C to be the series reactance
    L = Xp / ω # we choose L to be the parallel reactance
    return L, C

f = 98e6 # Frequency in the middle of the FM band

for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    L, C = calc_L_match(f, 50, Rin[i])
    print("\tInductance: {:.2e} H".format(L))
    print("\tCapacitance: {:.2e} F".format(C))

For Ic = 1.00 mA
	Load resistance: 50.00 ohm
	Source resistance: 26.00 ohm
	Q: 0.96
	Inductance: 8.45e-08 H
	Capacitance: 6.50e-11 F
For Ic = 2.00 mA
	Load resistance: 50.00 ohm
	Source resistance: 13.00 ohm
	Q: 1.69
	Inductance: 4.81e-08 H
	Capacitance: 7.40e-11 F
For Ic = 3.00 mA
	Load resistance: 50.00 ohm
	Source resistance: 8.67 ohm
	Q: 2.18
	Inductance: 3.72e-08 H
	Capacitance: 8.58e-11 F


## Simulation
<figure>
    <img src="./doc/simulation/4_input_match.png" width="100%">
    <figcaption>Input matching network for bias settings of 1mA, 2mA and 3mA.</figcaption>
</figure>

Remark that the simulated values differ quite a bit from the calculated values.  

# 6. Output impedance
## Output capacitance

In [18]:
# Frequency pole due to capacitance on the output

for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    f_p2 = 1 / (2* math.pi * Cµ * Rout[i]) # 7.2.4.1 : Common-base amplifier frequency response, p. 514
    print("Cutoff frequency due to capacitance at the output: {:.2f} MHz".format(f_p2 / 1e6))

For Ic = 1.00 mA
Cutoff frequency due to capacitance at the output: 49.99 MHz
For Ic = 2.00 mA
Cutoff frequency due to capacitance at the output: 99.97 MHz
For Ic = 3.00 mA
Cutoff frequency due to capacitance at the output: 149.96 MHz


## Simulation
<figure>
    <img src="./doc/simulation/5_output_impedance.png" width="1000"/>
    <figcaption>Output impedance of the amplifier and the effect of the output capacitance, causing roll-off at high frequencies.</figcaption>
</figure>

# 7. Image filter
## Calculation
see [Image filter](Filters.ipynb)

## Simulation
<figure>
    <img src="./doc/simulation/6_image_filter.png" width="100%">
    <figcaption>Image filter for bias settings of 1mA, 2mA and 3mA.</figcaption>
</figure>

It takes a bit of tuning to get the Q factor right.  Increasing the inductance value, lowers the Q factor.  The parallel capacitor then needs to be tuned to get the correct resonance frequency.  

# 8. Output Matching network
## Calculation

In [19]:
for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    L, C = calc_L_match(f, Rout[i], 50)
    print("\tInductance: {:.2e} H".format(L))
    print("\tCapacitance: {:.2e} F".format(C))

For Ic = 1.00 mA
	Load resistance: 3184.05 ohm
	Source resistance: 50.00 ohm
	Q: 7.92
	Inductance: 6.53e-07 H
	Capacitance: 4.10e-12 F
For Ic = 2.00 mA
	Load resistance: 1592.03 ohm
	Source resistance: 50.00 ohm
	Q: 5.55
	Inductance: 4.66e-07 H
	Capacitance: 5.85e-12 F
For Ic = 3.00 mA
	Load resistance: 1061.35 ohm
	Source resistance: 50.00 ohm
	Q: 4.50
	Inductance: 3.83e-07 H
	Capacitance: 7.22e-12 F


The Q-factor should be less than 6, otherwise the bandwidth will be too narrow for the whole FM band.

## Simulation
<figure>
    <img src="./doc/simulation/7_output_match.png" width="100%">
    <figcaption>Output matching network for bias settings of 1mA, 2mA and 3mA.</figcaption>
</figure>

# Power gain
## Derivation
<img src="./doc/simulation/two-port-network.png" width="600"/>

The unloaded voltage gain of the amplifier without info over input and output impedance is not meaningful.  To take these into account, power gain calculation is more useful.  

The voltage gain of the terminated amplifier is the voltage on R_L divided by the voltage on R_in.  Matching networks will be inserted at the input and at the output to match the input and output impedance to 50 Ohm.  
The total gain of the amplifier, including the matching networks is calculated below.

$$ G_{T}=A_{Vloaded}^{2}*\frac{R_{in}}{R_{out}} $$
when the input and output of the amplifier is matched to its load:
$$ A_{Vloaded} = \frac{A_{Vunloaded}}{2} $$
In case of a common base amplifier
$$ A_{Vunloaded} = -g_{m}*R_{C} $$
$$ Rin = \frac{1}{g_{m}} $$
$$ Rout = R_{C} $$
Putting it all together, the transducer power gain for a common base amplifier is given by:
$$ G_{T} = \frac{g_{m}*R_{C}}{4} $$

## Calculation

In [20]:
Av_loaded = [x / 2 for x in Av_no_load]   # if Rout = R_L
Gt_calc1 = [(x_Av_loaded**2) * y_Rin / z_Rout for x_Av_loaded, y_Rin, z_Rout in zip(Av_loaded, Rin, Rout)] # Transducer gain
Gt_calc2 = [x_gm * y_Rout / 4 for x_gm, y_Rout in zip(gm, Rout)] # Transducer gain
for i in range(len(Ic)):
    print("For Ic = {:.2f} mA".format(Ic[i]*1e3))
    print("\tTransducer gain (calculation method 1): {:.2f}dB".format(10 * math.log10(Gt_calc1[i]))) 
    print("\tTransducer gain (calculation method 2): {:.2f}dB".format(10 * math.log10(Gt_calc2[i])))


For Ic = 1.00 mA
	Transducer gain (calculation method 1): 14.86dB
	Transducer gain (calculation method 2): 14.86dB
For Ic = 2.00 mA
	Transducer gain (calculation method 1): 14.86dB
	Transducer gain (calculation method 2): 14.86dB
For Ic = 3.00 mA
	Transducer gain (calculation method 1): 14.86dB
	Transducer gain (calculation method 2): 14.86dB


## Simulation
<figure>
    <img src="./doc/simulation/8_wideband_gain.png" width="100%">
    <figcaption>Wideband gain of the completed amplifiers.</figcaption>
</figure>

As can be seen, the frequencies higher than our target frequency are not attenuated.  This makes this filter less desirable for high-side injection.  We will have to tune our local oscillator to a lower frequency than the target frequency, i.e. use low-side injection.
It might be better to change the input matching network and use the low-pass version instead.  We already have a 10nF coupling capacitor at the input.

[Varicap tricks](https://www.pi4raz.nl/index.php/2008/12/21/varicaps-tips-tricks/)

# IP3 : Third Order Intercept Point

# Selectivity

# References
* [MegawattKS : Radio Design 101 - Episode 2 - Impedance Matching - Part 2](https://youtu.be/ZWBem8GTzNs)
* [MegawattKS : Radio Design 101 - Episode 3 - RF Amplifiers](https://youtu.be/UUlqW-vSq9M)